This was the initial approach that I tried.
The main idea was comparing every post with every other post with at least one common word and using a simple ML model to do binary classification about them being match or not.
A dataset was generated for such pairs after extracting features.
The features included:
1. EfficientNetB1 cosine similarity.
1. Jaccard Distance.
1. Jaccard Index of the half of text which is more similar.
1. Fuzzy Ratio

A Decision Tree model with 7 depth was trained on that data.

A lot of text preprocessing was also done which also involved creating dataset of languages used on Shopee.

In [ ]:
import numpy as np 
import pandas as pd
import cv2
import gc
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
import tqdm.notebook as tq
from fuzzywuzzy import fuzz
import random
import math
import re
import time
import operator
import pickle
import warnings
warnings.filterwarnings('ignore')
import os

In [ ]:
def stringReplacer(st):
    st = st.replace('(','').replace(')','')
    st = st.replace('[','').replace(']','')
    st = st.replace('!','').replace(')','')
    st = st.replace('#',' ')
    st = st.replace('-',' ')
    st = st.replace(',',' ')
    st = st.replace('\\',' ')
    st = st.replace('/',' ')
    st = st.replace('|',' ')
    st = st.replace(' IN ',' ')
    st = st.replace(' THE ',' ')
    st = st.replace('"',' ')
    st = st.replace(' X ',' ')
    st = st.replace('   ','  ')
    st = st.replace('  ',' ')
    return st

def cleanSlashX(st):
    st2 = ''
    i=0

    while i<len(st):
        if st[i:i+2]=='\X':
            i+=4
        else:
            st2 += st[i]
            i+=1
    return st2

def cleanup_text(text):
    text = cleanSlashX(text)
    text = stringReplacer(text)
    text = "".join([c if ord(c) < 128 else "" for c in text]).strip() 
    return text

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')
test['title'] = test['title'].str.upper()
test['title'] = test['title'].apply(cleanup_text)

In [ ]:
ewf = pd.read_csv('../input/english-word-frequency/unigram_freq.csv')
ewf['word']=ewf['word'].str.upper()
#engAW = set(ewf['word'].iloc[:30001])
engAW = set(ewf['word'].iloc[:40001])
fname = '../input/indonesian-and-malaysian-common-words-list/indonesian.txt'
with open(fname) as f:
    indoAW = set(f.read().upper().splitlines())
fname = '../input/indonesian-and-malaysian-common-words-list/malay.txt'
with open(fname) as f:
    #malayAW = set(f.read().upper().splitlines()[:20000])
    malayAW = set(f.read().upper().splitlines()[:30000])

In [ ]:
def getLangData(langname,datapath):
    langEng = pd.read_csv(datapath)
    langEng[langname]=langEng[langname].str.upper()
    langEng['English']=langEng['English'].str.upper()
    langEngDict = {}
    for i in range(langEng.shape[0]):
        langEngDict[  langEng[langname].iloc[i]  ] = langEng['English'].iloc[i]
    langWords = set(langEng[langname])
    return (langEngDict,langWords)

In [ ]:
indoEngDict,indoWords1K = getLangData('Indonesian','../input/indonesian-english-1kwords/indonesian_eng.csv')
filipEngDict,filipWords1K = getLangData('Filipino','../input/indonesian-english-1kwords/filipino_eng.csv')
malayEngDict,malayWords1K = getLangData('Malay','../input/indonesian-english-1kwords/malaysian_eng.csv')
portuEngDict,portuWords1K = getLangData('Portugese','../input/indonesian-english-1kwords/portugese_eng.csv')
thaiEngDict,thaiWords1K = getLangData('Thai','../input/indonesian-english-1kwords/thai_eng.csv')
vietEngDict,vietWords1K = getLangData('Vietnamese','../input/indonesian-english-1kwords/vietnamese_eng.csv')
langEng = pd.read_csv('../input/indonesian-english-1kwords/indonesian_eng.csv')
langEng['English']=langEng['English'].str.upper()
engWords1K = set(langEng['English'])
filipAW = filipWords1K
portuAW = portuWords1K
germEngDict,germWords1K = getLangData('German','../input/indonesian-english-1kwords/german_eng.csv')
germAW = germWords1K

In [ ]:
langFilip = pd.read_csv('../input/indonesian-english-1kwords/filipino_eng.csv')
langFilip['Filipino']=langFilip['Filipino'].str.upper()
filipSW = set(langFilip['Filipino'].iloc[:51])

langIndo = pd.read_csv('../input/indonesian-english-1kwords/indonesian_eng.csv')
langIndo['Indonesian']=langIndo['Indonesian'].str.upper()
indoSW = set(langIndo['Indonesian'].iloc[:51])

langMalay = pd.read_csv('../input/indonesian-english-1kwords/malaysian_eng.csv')
langMalay['Malay']=langMalay['Malay'].str.upper()
malaySW = set(langMalay['Malay'].iloc[:51])

langPortu = pd.read_csv('../input/indonesian-english-1kwords/portugese_eng.csv')
langPortu['Portugese']=langPortu['Portugese'].str.upper()
portuSW = set(langPortu['Portugese'].iloc[:51])

langEng = pd.read_csv('../input/indonesian-english-1kwords/malaysian_eng.csv')
langEng['English']=langEng['English'].str.upper()
engSW = set(langEng['English'].iloc[:51])
engSW.remove('IN')
engSW.add('MY')
engSW.add("IT'S")
    
allSW = engSW.union(indoSW,malaySW,portuSW,filipSW)
#allAW = engAW.union(indoAW,malayAW,portuAW,filipAW)
allAW = engAW.union(indoAW,malayAW,portuAW,filipAW,germAW)

In [ ]:
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    if a.shape != b.shape:
        raise RuntimeError("array {} shape not match {}".format(a.shape, b.shape))
    if a.ndim==1:
        a_norm = np.linalg.norm(a)
        b_norm = np.linalg.norm(b)
    elif a.ndim==2:
        a_norm = np.linalg.norm(a, axis=1, keepdims=True)
        b_norm = np.linalg.norm(b, axis=1, keepdims=True)
    else:
        raise RuntimeError("array dimensions {} not right".format(a.ndim))
    similiarity = np.dot(a, b.T)/(a_norm * b_norm) 
    return similiarity

def cosine_similarity2(a, b):
    a = np.array(a)
    b = np.array(b)
    return dot(a, b)/(norm(a)*norm(b))

def dot_product2(v1, v2):
    return sum(map(operator.mul, v1, v2))

def cosine_similarity3(v1, v2):
    prod = dot_product2(v1, v2)
    len1 = math.sqrt(dot_product2(v1, v1))
    len2 = math.sqrt(dot_product2(v2, v2))
    return prod / (len1 * len2)

In [ ]:
#from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import EfficientNetB1

def findDifference(f1, f2):
    return np.linalg.norm(f1-f2)
 
def cosine_distance(a, b):
    if a.shape != b.shape:
        raise RuntimeError("array {} shape not match {}".format(a.shape, b.shape))
    if a.ndim==1:
        a_norm = np.linalg.norm(a)
        b_norm = np.linalg.norm(b)
    elif a.ndim==2:
        a_norm = np.linalg.norm(a, axis=1, keepdims=True)
        b_norm = np.linalg.norm(b, axis=1, keepdims=True)
    else:
        raise RuntimeError("array dimensions {} not right".format(a.ndim))
    similiarity = np.dot(a, b.T)/(a_norm * b_norm) 
    dist = 1. - similiarity
    return dist

class FeatureExtractor_EffNetB1:
    def __init__(self):
        
        #base_model = EfficientNetB3(weights='../input/tfkeras-efficientnet-weights/efficientnetb3_notop.h5',\
        #                            include_top=False, pooling='avg', input_shape=(300,300,3))
        base_model = EfficientNetB1(weights='../input/tfkeras-efficientnet-weights/efficientnetb1_notop.h5',\
                                    include_top=False, pooling='avg', input_shape=(240,240,3))
        self.model = base_model
    
    def extract(self, img):
        #h1,w1 = (img.shape[0],img.shape[1])
        #img = img[ int(h1*0.2):int(h1*0.8) , int(w1*0.2):int(w1*0.8) , : ]
        #img = cv2.resize(img, (300,300), interpolation = cv2.INTER_AREA)
        img = cv2.resize(img, (240,240), interpolation = cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)
    
    def extract_batch(self,x):
        features = self.model.predict(x)
        normi = np.linalg.norm(features,axis=1)
        features = features / normi[:,None]
        return features

modelEffNetB1 = FeatureExtractor_EffNetB1()

In [ ]:
def getTermMatches(l1,l2):
    matches = []
    l1 = list(set(l1))
    l2 = list(set(l2))
    l2C = ''.join(l2)
    for t1 in l1:
        inset1 = set(t1[1:-1])
        if t1 in l2:
            matches.append(t1)
        else:
            if t1 in l2C:
                matches.append(t1)
            else:
                for t2 in l2:
                    inset2 = set(t2[1:-1])
                    if len(t1)>3 and len(t2)>3:
                        if t1[0]==t2[0] and t1[-1]==t2[-1] and len(inset1 ^ inset2)<=2 and len(inset1.intersection(inset2))>3 :
                            matches.append(t1)
                            break
  
    matches = set(matches)
    return matches

In [ ]:
def breakPostfixQuants(ttl,psfx):
    ttlB = []
    qTpls = []
    for wrd in ttl:
        found = False
        for fx in psfx:
            if wrd[-len(fx):]==fx:
                n = len(wrd)-len(fx)
                if n > 0:
                    prv = wrd[:n]
                    if prv.isnumeric():
                        found = True
                        ttlB.append(prv)
                        ttlB.append(fx)
        if not found:
            ttlB.append(wrd)
            
    psfx = set(psfx)
    for i,wrd in enumerate(ttlB):
        if i>0:
            if wrd in psfx:
                prv = ttlB[i-1]
                if prv.isnumeric():
                    qTpls.append( (prv,wrd) )
                    
    for tpl in qTpls:
        ttlB.remove(tpl[0])
        ttlB.remove(tpl[1])
            
    return (ttlB,qTpls)

In [ ]:
def getTermMisMatches(l1,l2):
    l1 = set(l1)
    l2 = set(l2)
    mismatches = list(l1 ^ l2)
    return set(mismatches)

In [ ]:
def preprocessTitle(ttl):
    #ttl = cleanup_text(ttl)
    #ttl = stringReplacer(ttl)
    tl = ttl.split(' ')
    tl = [wd for wd in tl if wd not in allSW]
    psfx = ['ML','LTR','KG','GM','GR','GRM','GRAM','PC','PCS','X','W','V','IN','G','CM','M','OZ']
    tl,qtpl = breakPostfixQuants(tl,psfx)
    btl = set(zip(tl,tl[1:]))
    return (tl,qtpl,btl)

In [ ]:
def tokenizeTitle(ttl):
    tl = ttl.split(' ')
    tl = [wd for wd in tl if wd not in allSW]
    return set(tl[:30])


In [ ]:
def getUnigramMatchings(tl1,tl2,qtpl1,qtpl2):

    qNums = set([ q[0] for q in qtpl1 ] + [ q[0] for q in qtpl2 ])
    
    TF = getTermMatches(tl1,tl2)
    
    #TF = [ m for m in TF if any(map(str.isdigit,m))==False ]
    
    TF = [wd for wd in TF if len(wd)>0]
    
    DF = []
    NF = []
    for wd in TF:
        if wd in allAW:
            DF.append(wd)
        else:
            NF.append(wd)
    TN = getTermMisMatches(tl1,tl2)
    
    TN = [ m for m in TN if m not in qNums ]
    
    #TN = [ m for m in TN if any(map(str.isdigit,m))==False ]
    
    TN = [wd for wd in TN if len(wd)>0] #0
    
    DN = []
    NN = []
    for wd in TN:
        if wd in allAW:
            DN.append(wd)
        else:
            NN.append(wd)
            
    mDict = {'TF':TF,'DF':DF,'NF':NF,'TN':TN,'DN':DN,'NN':NN,'T1':tl1,'T2':tl2,'Q1':qtpl1,'Q2':qtpl2 }
    
    return mDict

In [ ]:
testMode = False

In [ ]:
def stringReplacer(st):
    st = st.replace('(','').replace(')','')
    st = st.replace('[','').replace(']','')
    st = st.replace('!','').replace(')','')
    st = st.replace('#',' ')
    st = st.replace('-',' ')
    st = st.replace(',',' ')
    st = st.replace('\\',' ')
    st = st.replace('/',' ')
    st = st.replace('|',' ')
    st = st.replace(' IN ',' ')
    st = st.replace(' THE ',' ')
    st = st.replace('"',' ')
    st = st.replace(' X ',' ')
    st = st.replace('   ','  ')
    st = st.replace('  ',' ')
    return st

def cleanSlashX(st):
    st2 = ''
    i=0

    while i<len(st):
        if st[i:i+2]=='\X':
            i+=4
        else:
            st2 += st[i]
            i+=1
    return st2

def cleanup_text(text):
    text2 = stringReplacer(text)
    text3 = "".join([c if ord(c) < 128 else "" for c in text2]).strip() 
    return cleanSlashX(text2)

In [ ]:
if testMode:
    train = pd.read_csv('../input/shopee-product-matching/train.csv')
    train['title'] = train['title'].str.upper()
    train['title'] = train['title'].apply(cleanup_text)
    train = train.sample(frac = 1,random_state=0).reset_index(drop=True) 
    df = train.iloc[:1000]
    #df = df.append(df)
    #df = df.reset_index(drop=True)
    bPath = '../input/shopee-product-matching/train_images'
    print(df)
    
else:
    df = test
    bPath = '../input/shopee-product-matching/test_images'

In [ ]:
startT = time.time()
df['tokens'] = df['title'].apply(tokenizeTitle)
endT = time.time()
print('Time Elapsed : ', (endT - startT)  )

In [ ]:
def matchesAny(l):
    global tkns
    return len(l.intersection(tkns))>1

def matchesAny2(l):
    global tkns,ttlC
    return len(l[0].intersection(tkns))>1 or (l[1]==ttlC)

def matchesAny3(l):
    global tkns,ttlC,phsh
    return len(l[0].intersection(tkns))>1 or l[1]==ttlC or l[2]==phsh 

In [ ]:
def jiCheck(l):
    global tkns
    return (len(l.intersection(tkns)) / len( l.union(tkns) )) > 0.1

In [ ]:
validVD = {}
for i in tq.tqdm(range(  int(df.shape[0])  )):
    validVD[i] = []
    tkns = df['tokens'].iloc[i]
    ttlC = df['title'].iloc[i]
    phsh = df['image_phash'].iloc[i]
    mL = set(df[ df['tokens'].apply(matchesAny) ].index)
    #mL = list(df[ df['tokens'].apply(matchesAny2) ].index)
    #mL = set(df[ df['tokens'].apply(matchesAny3) ].index)
    validVD[i] = mL

In [ ]:
df.drop(columns=['tokens'],inplace=True)

In [ ]:
pptDict = {}
embNList = []

for i in tq.tqdm( range(df.shape[0]) ):
    
    ttl1 = df['title'].iloc[i]

    tl1,qtpl1,btl1 = preprocessTitle(ttl1)
    
    imgP = df['image'].iloc[i]
    img = cv2.imread(  os.path.join(bPath,imgP) )
    embNList.append( modelEffNetB1.extract(img) )
    
    pptDict[i] = (tl1,qtpl1,btl1)

In [ ]:
imgEmbs_np = np.array(embNList)
del embNList
gc.collect()

In [ ]:
startT = time.time()
nbrs = NearestNeighbors(n_neighbors=min(50,df.shape[0]), algorithm='ball_tree').fit(imgEmbs_np)
distances, indices = nbrs.kneighbors(imgEmbs_np)
endT = time.time()
print('Time Elapsed : ',endT - startT)

In [ ]:
n = 0
for i in tq.tqdm(range(df.shape[0])):
    nbrInds = indices[i,1:]
    nbrDists = distances[i,1:]
    nbrVInds = []
    for ni,d in enumerate(nbrDists):
        if d<0.5:
            nbrVInds.append( nbrInds[ni] )
        else:
            break
    
    nbrVInds = set(nbrVInds)
    vldInds = validVD[i]
    unn = vldInds.union(nbrVInds)
    if len(unn) > len(vldInds):
        validVD[i] = unn
        n += 1
        
print(n*100/df.shape[0])

del distances
del indices
gc.collect()

In [ ]:
def getMaxFuzzyMatch(ttl1,ttl2):
    ttlS = ttl1
    ttlB = ttl2
    if len(ttlB)<len(ttlS):
        ttlS,ttlB = ttlB,ttlS
    
    blockL = ttlB[:len(ttlS)]
    blockR = ttlB[-len(ttlS):]
    
    fzL = fuzz.ratio(ttlS,blockL)
    fzR = fuzz.ratio(ttlS,blockR)
    fzMax = max(fzL,fzR)
    return fzMax

In [ ]:
def getMaxRI(tl1,tl2):
    ttlS = tl1
    ttlB = tl2
    if len(ttlB)<len(ttlS):
        ttlS,ttlB = ttlB,ttlS
    
    blockL = ttlB[:len(ttlS)]
    blockR = ttlB[-len(ttlS):]

    ri1 =  (len(set(blockL).union(ttlS)) - len( set(blockL).intersection(set(ttlS)) )) / len(set(blockL).union(ttlS))
    ri2 = (len(set(blockR).union(ttlS)) - len( set(blockR).intersection(set(ttlS)) )) / len(set(blockR).union(ttlS))
    return max(ri1,ri2)

In [ ]:
def getMaxJI(tl1,tl2):
    ttlS = tl1
    ttlB = tl2
    if len(ttlB)<len(ttlS):
        ttlS,ttlB = ttlB,ttlS
    
    blockL = ttlB[:len(ttlS)]
    blockR = ttlB[-len(ttlS):]

    ji1 = len( set(blockL).intersection(set(ttlS)) ) / len(set(blockL).union(ttlS))
    ji2 = len( set(blockR).intersection(set(ttlS)) ) / len(set(blockR).union(ttlS))
    return max(ji1,ji2)

In [ ]:
def quantMismatched(Q1,Q2):
    #psfx = ['ML','LTR','KG','GM','GR','GRM','GRAM','PC','PCS','X','W','V','IN','G','CM','M','OZ']
    #Q1 = breakPostfixQuants(TL1,psfx)[1]
    #Q2 = breakPostfixQuants(TL2,psfx)[1]
    Q1 = set([q[0] for q in Q1 ])
    Q2 = set([q[0] for q in Q2 ])
    nf = False
    if len(Q1)>0 and len(Q2)>0:
        QL = Q1
        QS = Q2
        if len(QS)>len(QL):
            QS = Q1
            QL = Q2
        nf = False
        for q in QS:
            if q not in QL:
                nf = True
                break
    return nf

In [ ]:
doRec = False

In [ ]:
def getWordCountsS(potDocs):
    wcountDict = {}
    for doc in potDocs:
        tkns = doc.split(' ')
        for w in tkns:
            if w in wcountDict:
                wcountDict[w]+=1
            else:
                wcountDict[w]=1
    return wcountDict

def getTFIDFSim(querDoc,potDocs):
    potDocs = [querDoc] + potDocs
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(potDocs)
    tfidfDVecs = X.todense()
    tfidSims = cosine_similarity(tfidfDVecs[0],tfidfDVecs)[0][1:]
    return tfidSims

def getTFIDFSimPair(tl1,tl2, keyLPos, potWordCounts):
    
    vlen = len(keyLPos)
    
    vec1 = [0.0]*vlen
    for w in tl1:
        pw = potWordCounts[w]
        if pw>0:
            vec1[ keyLPos[w] ] = 1.0 / pw
        
    vec2 = [0.0]*vlen
    for w in tl2:
        pw = potWordCounts[w]
        if pw>0:
            vec2[ keyLPos[w] ] = 1.0 / pw 
        
    try:
        cos_sim = cosine_similarity3(vec1,vec2)
    except:
        cos_sim = 0
    
    return cos_sim
    
def getWtdJI(tl1,tl2,wCntDict):
    aIbt = list(set(tl1).intersection(set(tl2)))
    aUbt = list(set(tl1).union(set(tl2)))
    iCnt = 0
    uCnt = 0
    for w in aIbt:
        wc = wCntDict[w]
        if wc>0:
            iCnt += (1.0/wc)
    for w in aUbt:
        wc = wCntDict[w]
        if wc>0:
            uCnt += (1.0/wc)
    wJI = iCnt / uCnt
    return wJI

In [ ]:
model = pickle.load(open('../input/shopee-decisiontrees/mxdt7_smxjfzri.sav', 'rb'))

In [ ]:
eva = True

In [ ]:
n = df.shape[0]
for i in tq.tqdm(range(n)):
    qHash = df['image_phash'].iloc[i]
    hashSet = set(df[ df['image_phash'] == qHash  ].index)
    qTitle = df['title'].iloc[i]
    titleSet = set(df[ df['title'] == qTitle  ].index)
    validVD[i] = validVD[i].union(hashSet)
    validVD[i] = validVD[i].union(titleSet)

In [ ]:
if testMode and False:
    n = df.shape[0]
    m = 0
    for i in tq.tqdm(range(n)):
        lgp = df['label_group'].iloc[i]
        gdf = df[ df['label_group']==lgp ]
        gdfi = set(gdf.index)
        if len(gdfi)>1:
            
            vld = validVD[i]
            
            sHash = set(df[ df['image_phash'] == df['image_phash'].iloc[i]  ].index)
            
            vld = vld.union(sHash)
            
            mt = vld.intersection(gdfi)

            unfounds = []
            for mi in gdfi:
                if mi not in vld:
                    unfounds.append(mi)


    print( round(m*100/n,3) )

In [ ]:
resultDict = {}

featsDict = {'JI':[],'WJI':[],'SE3':[],'QMM':[],'LR':[],'BF':[],'UF':[],\
              'MXFZ':[],'MXJI':[],'WMXJI':[], 'IND':[], 'BJI':[], 'TL1':[], 'TL2':[],\
              'FZ':[],'IDFS':[], 'isDup':[] }

res = {}
res['TP'] = 0
res['FP'] = 0
res['TN'] = 0
res['FN'] = 0
res['correct'] = 0
res['total'] = 0
res['trueP'] = 0
res['falseP'] = 0
res['trueT'] = 0
res['falseT'] = 0

dprobs = []

for i in tq.tqdm( range(df.shape[0])  ):
    
    postID1 = df['posting_id'].iloc[i]
    ttl1 = df['title'].iloc[i]
    phash1 = df['image_phash'].iloc[i]
    
    resultDict[postID1] = set()
    
    tl1,qtpl1,btl1 = pptDict[i]
    
    validSet = list(validVD[i])
    
    """
    potDocs = [ df['title'][pI] for pI in validSet ]
    potWordCounts = getWordCountsS(potDocs)
    keyList = list(potWordCounts.keys())
    keyLPos = {}
    for ki,kw in enumerate(keyList):
        keyLPos[kw]=ki
    """
    
    for nj,j in enumerate(validSet):
        
        postID2 = df['posting_id'].iloc[j]
        ttl2 = df['title'].iloc[j]
        phash2 = df['image_phash'].iloc[j]
        
        if testMode and eva:
            lbl1 = df['label_group'][i]
            lbl2 = df['label_group'][j]
            duplicTruth = lbl1==lbl2
        
        duplicPred = True
        
        if postID1 != postID2:

            if postID2 in resultDict:
                if postID1 in resultDict[postID2]:
                    duplicPred = True
                else:
                    duplicPred = False
            else:

                duplicPred = True

                if phash1!=phash2:
                    
                    
                    f1 = imgEmbs_np[i]
                    f2 = imgEmbs_np[j]
                    sim = findDifference(f1, f2)

                    tl2,qtpl2,btl2 = pptDict[j]
                    
                    
                    qMM = quantMismatched(qtpl1,qtpl2)
                    if qMM:
                        qMM=1
                    else:
                        qMM=0
                    
                    
                    #wJI = getWtdJI(tl1,tl2,potWordCounts)
                    
                    #wJIMax = getMaxWtdJI(tl1,tl2,potWordCounts)
                    
                    #tfidSim = tfidfSims[nj]
                    
                    #tfidSim = getTFIDFSimPair(tl1,tl2, keyLPos, potWordCounts)
                    
                    aUbU = len( set(tl1).union(set(tl2)) )
                    aUbB = len( btl1.union(btl2) )
                    aIbU = len( set(tl1).intersection(set(tl2)) )
                    aIbB = len( btl1.intersection(btl2) )
                    
                    Jid = aIbU / aUbU
                    Rid = 1 - Jid
                    
                    """
                    
                    if aUbB>0:
                        BJid = aIbB / aUbB
                    else:
                        BJid = 0
                    """
                    #mDict = getUnigramMatchings(tl1,tl2,qtpl1,qtpl2)

                    #lenRatio = float(max( len(ttl1),len(ttl2) )) / float(min( len(ttl1),len(ttl2) ) )
                        
                    fz = fuzz.ratio(ttl1,ttl2)    
                    
                    #fzMax = getMaxFuzzyMatch(ttl1,ttl2)
                    
                    jiMax = getMaxJI(tl1,tl2)
                    
                    duplicPred = False
                    
                    #featVec = np.array([[sim,tfidSim,wJI,fzMax]])
                    featVec = np.array([[sim,jiMax,fz,Rid]])
                    duplicPred = model.predict( featVec )[0] == 1
                    
                    #duplicPred = qMM==0 and (sim<0.63 or tfidSim>0.48)
                    
                    duplicPred = duplicPred and qMM==0
                    
                    #proba = model.predict_proba( featVec )
                    #dupProb = proba[0,1]
                    """
                    ndupProb = proba[0,0]
                    if duplicPred:
                        dprobs.append(dupProb)
                    """
                        
                    #duplicPred = duplicPred and dupProb>0.6
                    
                    #duplicPred = duplicPred or (Jid>0 and sim<0.15)
                    
                    #duplicPred = duplicPred or (jiMax>0 and sim<0.25)
                    #duplicPred = duplicPred or (wJI>0 and sim<0.25)
                    
                    #duplicPred = ( (sim < 1.24 and Jid > 0.7) or (sim < 0.8 and Jid > 0.37) or (sim < 0.5 and Jid > 0) or (sim<0.9 and fz>70))\
                    #and (len(mDict['TN'])<21) and (lenRatio<13) #** 0.685 
                    
                    
                    ss=0
                    
                    if ttl1==ttl2:
                        duplicPred = True
                        ss=1
               
                    #################
                    if testMode and eva:
                        pass
                        #lbl1 = df['label_group'][i]
                        #lbl2 = df['label_group'][j]
                        #duplicTruth = lbl1==lbl2
                        
                        """
                        
                        featsDict['IND'].append( (i,j) )
                        featsDict['TL1'].append( ttl1 )
                        featsDict['TL2'].append( ttl2 )
                        featsDict['JI'].append(Jid)
                        featsDict['WJI'].append(wJIMax)
                        featsDict['BJI'].append(BJid)
                        featsDict['SE3'].append(sim)
                        featsDict['QMM'].append(qMM)
                        featsDict['LR'].append(lenRatio)
                        featsDict['UF'].append(aIbU)
                        featsDict['BF'].append(aIbB)
                        featsDict['FZ'].append(fz)
                        featsDict['MXFZ'].append(fzMax)
                        featsDict['MXJI'].append(jiMax)
                        featsDict['WMXJI'].append(wJIMax)
                        featsDict['IDFS'].append(tfidSim)
                        

                        if duplicTruth:
                            featsDict['isDup'].append(1)
                            
                        else:
                            featsDict['isDup'].append(0)
                            
                        """

                        
                    
                    
                    #################
                        

                else:
                    duplicPred = True
                    
        if testMode and eva:
            if duplicTruth==True and duplicPred==True:
                res['TP'] += 1

            if duplicTruth==True and duplicPred==False:
                res['FN'] += 1

            if duplicTruth==False and duplicPred==True:
                res['FP'] += 1

            if duplicTruth==False and duplicPred==False:
                res['TN'] += 1

            if duplicTruth == duplicPred:
                res['correct'] += 1
            if duplicPred==True:
                res['trueP'] += 1
            else:
                res['falseP'] += 1
            if duplicTruth==True:
                res['trueT'] += 1
            else:
                res['falseT'] += 1
            res['total'] += 1
            

        if duplicPred:
            resultDict[postID1].add(postID2)

In [ ]:
posting_id = df['posting_id'].tolist()
matches = []

for i in posting_id:
    matchesL = list(resultDict[ i ])
    matchStr = ''
    for match in matchesL:
        matchStr += match+' '
    matchStr = matchStr[:-1]
    matches.append(matchStr)
    
sub_df = pd.DataFrame({'posting_id': posting_id, 'matches': matches})
sub_df.to_csv('submission.csv', index=False)

In [ ]:
if testMode:
    res['F1'] = res['TP'] / ( res['TP'] + 0.5*(res['FP']+res['FN']) )
    print(res['F1'] )
    res['Accr'] = res['correct'] * 100 / res['total']
    print(res)